<h3 style = "font-family: Cambria;"> Data </h3>

In [1]:
import pandas, numpy, gurobipy, datetime

price = pandas.read_csv("Price.csv", index_col = "Date")

index = numpy.array([])

for i in price.index:

    index = numpy.append(index, datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S"))

price = price.set_index(index)

print(price)

                     A         AA        AAL   AAN       AAON         AAP  \
2020-01-02   83.502022  20.890644  28.982893   NaN  32.463730  146.164062   
2020-01-03   82.161339  20.968666  27.548195   NaN  32.763054  146.173248   
2020-01-06   82.404205  20.481024  27.219410   NaN  32.704502  143.770935   
2020-01-07   82.656792  20.793118  27.119778   NaN  32.632919  142.065506   
2020-01-08   83.472877  19.954369  27.737495   NaN  32.541813  140.433411   
...                ...        ...        ...   ...        ...         ...   
2024-04-09  147.399994  36.820000  13.950000  7.61  89.580002   78.208359   
2024-04-10  144.160004  36.189999  13.410000  7.08  87.860001   77.699997   
2024-04-11  145.000000  36.230000  13.620000  7.14  87.610001   74.349998   
2024-04-12  140.729996  35.200001  13.150000  7.16  88.010002   70.500000   
2024-04-15  140.220001  36.570000  12.940000  7.09  87.139999   70.080002   

                  AAPL        ABBV      ABEV         ABT  ...       YORW  \

<h3 style = "font-family: Cambria"> Robust Estimators to Heavy Tails </h3>

In [2]:
returns = price.interpolate(method = "time").apply(numpy.log).apply(numpy.diff).dropna()

from scipy.stats import trim_mean

mat_m = numpy.array([trim_mean(returns[i], 0.05) for i in returns.columns])

print(pandas.DataFrame(mat_m, price.columns))

             0
A    -0.000133
AA   -0.000361
AAL  -0.000390
AAN   0.000039
AAON  0.001212
...        ...
ZIM  -0.001466
ZTO  -0.000224
ZTS   0.000003
ZWS   0.000864
ZYXI  0.000678

[1152 rows x 1 columns]


In [3]:
from sklearn.covariance import EmpiricalCovariance

variance = pandas.DataFrame(EmpiricalCovariance().fit(returns).covariance_)

variance.index = price.columns; variance.columns = price.columns

print(variance)

             A        AA       AAL       AAN      AAON       AAP      AAPL  \
A     0.000284  0.000201  0.000114  0.000118  0.000096  0.000144  0.000055   
AA    0.000201  0.001072  0.000307  0.000283  0.000231  0.000335  0.000121   
AAL   0.000114  0.000307  0.000572  0.000243  0.000140  0.000170  0.000107   
AAN   0.000118  0.000283  0.000243  0.001378  0.000180  0.000223  0.000080   
AAON  0.000096  0.000231  0.000140  0.000180  0.000441  0.000146  0.000088   
...        ...       ...       ...       ...       ...       ...       ...   
ZIM   0.000052  0.000348  0.000165  0.000126  0.000133  0.000212  0.000059   
ZTO   0.000063  0.000205  0.000054  0.000057  0.000023  0.000086  0.000051   
ZTS   0.000121  0.000177  0.000121  0.000120  0.000108  0.000079  0.000090   
ZWS   0.000140  0.000246  0.000195  0.000184  0.000167  0.000108  0.000094   
ZYXI  0.000100  0.000288  0.000170  0.000116  0.000114  0.000155  0.000099   

          ABBV      ABEV       ABT  ...          YORW       ZBH

<h3 style = "font-family: Cambria"> Data Cleaning </h3>

In [4]:
mat_k, mat_a, mat_b = pandas.read_csv("Clustering.csv", index_col = "Ticker"), {}, {}

for i in mat_k.columns:

    p = numpy.transpose(price)["2020-01-02 00:00:00"]

    a = numpy.array([i ** -1 if i != 0 else 0 for i in list(p.fillna(value = 0))])
    
    m = mat_m - 8.0e-6 - 1.5e-2 * a
    
    l = numpy.array(pandas.read_csv("Liquidity.csv", index_col = "Date").mean())

    mat_a[i] = {0: m, 1: l}; mat_b[i] = [1.2e-3, 5.0e-6]

In [5]:
mat_k, mat_c, mat_d = pandas.read_csv("Clustering.csv", index_col = "Ticker"), {}, {}

for i in mat_k.columns:

    temp_1 = numpy.zeros((numpy.max(mat_k[i]) + 1, len(mat_m))); temp_3 = numpy.ones(numpy.max(mat_k[i]) + 1) * 1.0e0

    temp_2 = numpy.zeros((numpy.max(mat_k[i]) + 1, len(mat_m))); temp_4 = numpy.ones(numpy.max(mat_k[i]) + 1) * 1.0e1
    
    for j in range(len(mat_k[i])):

        temp_1[list(mat_k[i])[j]][j] += 1

        temp_2[list(mat_k[i])[j]][j] += 1
    
    mat_c[i] = {0: temp_1, 1: temp_2}

    mat_d[i] = {0: temp_3, 1: temp_4}

<h3 style = "font-family: Cambria"> Portfolio Optimization </h3>

In [6]:
from gurobipy import GRB

def portfolio_optimization(mat_a, mat_b, mat_c, mat_d, mat_p, mat_s):

    result, mat_ones = {}, numpy.ones(len(mat_a[0]))

    portfolio = gurobipy.Model()

    variables = {"w": portfolio.addMVar(len(mat_a[0]), lb = 0), "b": portfolio.addMVar(len(mat_a[0]), vtype = GRB.BINARY)}

    portfolio.addConstr(mat_a[0] @ variables["w"] >= mat_b[0])

    portfolio.addConstr(mat_a[1] @ variables["w"] <= mat_b[1])

    portfolio.addConstr(mat_c[0] @ variables["b"] >= mat_d[0])

    portfolio.addConstr(mat_c[1] @ variables["b"] <= mat_d[1])

    portfolio.addConstr(mat_ones @ variables["w"] == 1); portfolio.addConstr(variables["w"] <= variables["b"])
            
    for k in range(len(mat_p)):

        if k == 0: portfolio.addConstr(variables["b"][k] == 0)
    
    portfolio.setObjective(variables["w"] @ numpy.array(mat_s) @ variables["w"], GRB.MINIMIZE)

    portfolio.update(); portfolio.optimize()

    if portfolio.status == 2:
                
        result["o"] = portfolio.getObjective().getValue()

        result["w"] = (variables["w"]).x

        return result
    
    return "Error"

<h3 style = "font-family: Cambria"> Result </h3>

In [7]:
vec_pr = numpy.transpose(price)["2020-01-02 00:00:00"]

vec_pr = numpy.array([i ** -1 if i != 0 else 0 for i in list(vec_pr.fillna(value = 0))])

result = {}

for i in mat_k.drop(["Trace Maximization"], axis = 1).columns:

    result[i] = portfolio_optimization(mat_a[i], mat_b[i], mat_c[i], mat_d[i], vec_pr, variance)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-09
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1170 rows, 2304 columns and 8065 nonzeros
Model fingerprint: 0x79450f91
Model has 664128 quadratic objective terms
Variable types: 1152 continuous, 1152 integer (1152 binary)
Coefficient statistics:
  Matrix range     [2e-10, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [9e-11, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-06, 1e+01]
Presolve removed 13 rows and 7 columns
Presolve time: 0.16s
Presolved: 1157 rows, 2297 columns, 8024 nonzeros
Presolved model has 662976 quadratic objective terms
Variable types: 1151 continuous, 1146 integer (1146 binary)
Found heuristic solution: objective 0.0000661

Root relaxation: objective 1.095616e-05, 315 iteration

In [25]:
dataframe, keys = {}, list(result.keys())

for i in range(len(keys)):

    r = mat_a[keys[i]][0] @ result[keys[i]]["w"]

    l = mat_a[keys[i]][1] @ result[keys[i]]["w"]
    
    dataframe[keys[i]] = {"Return": r, "Illiquidity": l, "Risk": result[keys[i]]["o"]}

print(pandas.DataFrame(dataframe))

             Lloyd's Algorithm  Frank-wolfe Algorithm
Return                0.001199               0.001199
Illiquidity           0.000003               0.000003
Risk                  0.000011               0.000011


In [30]:
weight = {i: result[i]["w"] for i in result}

weight = pandas.DataFrame(weight)

weight["Trace Maximization"] = weight["Lloyd's Algorithm"]

weight.to_csv("Portfolio.csv")